## Activate Function

A função de ativação é análoga à acumulação de potencial elétrico em neurônios biológicos que disparam quando um determinado potencial de ativação é atingido.

<img src="reports/activate_principles.png" align="center" height=auto width=80%/>

- As activate functions adicionam um fator de **não linearidade** nos modelos lineares

### Non-Linear Activate Functions
ao introduzir uma ativação não linear, a superfície de custo da rede neural deixa de ser uma hipébole convexa, tornando a otimização mais complicada. Isso causa algumas dificuldades de treinamento

https://matheusfacure.github.io/2017/07/10/problemas-treinamento/


### Activate Function Types
- Sigmoide
- TanH
- ReLU
- softmax

## **Sigmoide**

Como neurônios biológicos funcionam de foma binária (ativando vs não ativando), a função sigmoide é uma boa forma de modelar esse comportamento, já que assume valores apenas entre 0 (não ativação) e 1 (ativação). 

\sigma(x)=\frac{1}{1+e^x}  \quad \quad  \sigma'(x)=\sigma(x)(1-\sigma(x))

<img src="reports/graph_sigmoide.png" align="center" height=auto width=60%/>

Os sigmóides ainda são usados ​​como funções de saída para classificação binária, mas geralmente não são usados ​​em camadas ocultas. Uma versão multidimensional do sigmóide é conhecida como função softmax e é usada para classificação em várias classes.

#### Por que não usar sigmoide functions?
- Observando o gráfico é possível ver que sigmóides não são centralizados em zero; 
- se olharmos sua derivada, podemos ver que ela satura para valores de -5 até 5, ou seja, ela tende a zero.
- outro problema é que a derivada da função sigmoide é sempre < 1 (Vanishing Gradient Problem)
- No geral, a função logística praticamente desapareceu dos modelos modernos de redes neurais mais convencionais.

## **TanH**


Similar a função sigmoide, a função Tangente Hiperbólica (TanH) também tem um formato de ‘S’, mas varia de -1 a 1, em vez de 0 a 1 como na sigmoide. A TanH se aproxima mais da identidade, sendo assim uma alternativa mais atraente do que a sigmoide para servir de ativação às camadas ocultas das RNAs. A TanH sua derivada são dadas, respectivamente, por:

http://www.sciweavers.org/free-online-latex-equation-editor
\\[ tanh(x)=2\sigma(2x) - 1   \quad \quad  tanh'(x)=1 - tanh^2(x) ]\\

<img src="reports/graph_tanh.png" align="center" height=auto width=60%/>

- A derivada é maior, chegando ao máximo de 1 quando x=0. Por esse motivo, quando uma função sigmoidal precisa ser utilizada, recomenda-se a TanH no lugar da sigmoide.



## **ReLU Activate**
_rectified linear unit (ReLU)_

% <![CDATA[
ReLU(x)=max\{0, x\}   \quad \quad  ReLU'(x)=
	\begin{cases}
    	1, & \text{se } x\ge 0\\
    	0, & \text{c.c.}
	\end{cases} %]]>
    
<img src="reports/graph_relu.png" align="center" height=auto width=60%/>

Redes com a função ReLU são fáceis de otimizar, já que a ReLU é extremamente parecida com a função identidade. A única diferença é que a ReLU produz zero em metade do seu domínio. 


Quase todos os modelos de deep learning usam ReLU hoje em dia. No entanto, a ReLU deve ser usada apenas dentro de hidden layer de uma rede neural, e não para a camada de saída - que deve ser sigmóide para classificação binária, softmax para classificação multiclasse e linear para um problema de regressão.


To prevent neural network from wrong training, sigmoid should be used as the activation function in output layer. For the activation function of hidden layer, ReLU is better.

In [7]:
# ReLU
class Relu():
    def __init__(self):
        self._mask = None

    def forward(self, X):
        self._mask = (X <= 0)
        out = X.copy()
        out[self._mask] = 0

        return out

    def backward(self, d):
        d[self._mask] = 0
        dx = d

        return dx

#### Softmax Function
_Simply speaking, softmax function returns probability of the label_

S(y_i) = \frac{e^{y_i}}{\sum_j e^{y_j}}

softmax.png
- É uma função de scaling (normalized exponential function) que converte vetor de k-dimensões em um valor probabilistico entre (0, 1). 
- Usado no final de uma neural network
- Recomendado quando houve treinamento com cross-entropy
- Usar somente em **classificação multipla**

- Implemeantation:

In [18]:
logits = [2.0, 1.0, 0.1] 
import numpy as np 
exps = [np.exp (i) for i in logits]
print(exps)

[7.38905609893065, 2.718281828459045, 1.1051709180756477]


In [ ]:
import numpy as np

# Softmax function
def softmax(a):
    max_a = np.max(a)   # To prevent overflow
                        # Exponential could have too large value.
                        # max_a will be downscale the original value.
    exp_a = np.exp(a - max_a)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

x = np.random.rand(5,1)
y = softmax(x)

print("    {0:7}   {1:7}".format("X", "S(X)"))
for i in range(len(x)):
    print("    {0:7.6f}  {1:7.6f}".format(x[i][0], y[i][0]))
print("Sum {0:7.6f}  {1:7.6f}".format(np.sum(x), np.sum(y)))

In [ ]:
# Softmax-With-Loss
class SoftmaxWithLoss():
    def __init__(self):
        self._loss = None
        self._Y = None
        self._labels = None

    def _softmax(self, X):
        if X.ndim == 2:
            X = X.T
            X = X - np.max(X, axis=0)
            Y = np.exp(X) / np.sum(np.exp(X), axis=0)
            return Y.T

        # Protect from overflow
        X = X - np.max(x)
        return np.exp(X) / np.sum(np.exp(X))

    def _cross_entropy_error(self, Y, labels):
        if Y.ndim == 1:
            labels = labels.reshape(1, labels.size)
            Y = Y.reshape(1, Y.size)

        # If train data is one-hot encoded,
        #  translate it to answer label
        if labels.size == Y.size:
            labels = labels.argmax(axis=1)

        batch_size = Y.shape[0]

        log_val = np.log(Y[np.arange(batch_size), labels])
        return -np.sum(log_val) / batch_size

    def forward(self, X, labels):
        self._labels = labels
        self._Y = self._softmax(X)
        self._loss = self._cross_entropy_error(self._Y, self._labels)

        return self._loss

    def backward(self, d=1):
        batch_size = self._labels.shape[0]
        # If train data is one-hot encoded,
        #  translate it to answer label
        if self._labels.size == self._Y.size:
            dx = (self._Y - self._labels) / batch_size
        else:
            dx = self._Y.copy()
            dx[np.arange(batch_size), self._labels] -= 1
            dx = dx / batch_size

        return dx



### Summary About Activate and Cost Function

Existem funções de cost específicas que devem ser usadas em cada cenários, compatíveis com o tipo de output da neural network. 

A summary of the data types, distributions, output layers, and cost functions are given in the table below.

<img src="reports/distr_cost_output_layer.png" align="center" height=auto width=60%/>

| Problem  | Hidden Layer | Output Layer |
|----------|--------------|--------------|
| Linear   | Identity     | Identity     |
| binary classification | ReLU         | Sigmoid      |
| multi classification  | ReLU         | Softmax      |


 Por exemplo, usar o MSE em dados binários faz muito pouco sentido e, portanto, para dados binários, usamos a função de perda de entropia cruzada binária.
 
 

<img src="reports/comparation.png" align="center" height=auto width=60%/>

**ELU > leaky ReLU > ReLU >> tanh > sigmoide.**

**NOTE**:
- **Sigmoide** usar somente na layer de output em casos de **classificação binária.**
- **Softmax** usar somente na layer de output em casos de **classificação multipla**. 
- ReLU deve ser usada apenas dentro de hidden layer 


- [1] https://medium.com/data-science-bootcamp/understand-the-softmax-function-in-minutes-f3a59641e86d